In [1]:
from functional.backtester import Backtester as b
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from modeler_strats.universal_modeler import UniversalModeler
from transformer.adhoc_transforms import Adhoc as adhocs
from transformer.risk_transforms import Risk as risks
from transformer.analysis_transforms import Analysis as anas
from transformer.financial_transforms import Financial as financial

import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
from statistics import variance
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm

In [2]:
start_date = datetime(2023,1,1)
end_date = datetime.now()

In [3]:
market = Market()
sec = SEC()
fin_db = ADatabase("financial")
earnings_db = ADatabase("earnings")
dividends_db = ADatabase("dividends")
speculation_db = ADatabase("spec")
classification_db = ADatabase("spec_classification")
current_db = ADatabase("financial_current")
umod = UniversalModeler()

In [4]:
tyields = adhocs.tyields()
bench_returns = adhocs.spy_bench()

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
market.disconnect()

In [6]:
financial_factors = ['assets',
 'liabilitiesandstockholdersequity',
 'incometaxexpensebenefit',
 'retainedearningsaccumulateddeficit',
 'accumulatedothercomprehensiveincomelossnetoftax',
 'earningspersharebasic',
 'earningspersharediluted',
 'propertyplantandequipmentnet',
 'cashandcashequivalentsatcarryingvalue',
 'entitycommonstocksharesoutstanding',
 'weightedaveragenumberofdilutedsharesoutstanding',
 'weightedaveragenumberofsharesoutstandingbasic',
 'stockholdersequity']

In [7]:
dividend_factors = ['assets',
 'liabilitiesandstockholdersequity',
 'incometaxexpensebenefit',
 'retainedearningsaccumulateddeficit',
 'accumulatedothercomprehensiveincomelossnetoftax',
 'earningspersharebasic',
 'earningspersharediluted',
 'propertyplantandequipmentnet',
 'cashandcashequivalentsatcarryingvalue',
 'entitycommonstocksharesoutstanding',
 'weightedaveragenumberofdilutedsharesoutstanding',
 'weightedaveragenumberofsharesoutstandingbasic',
 'stockholdersequity',
"commonstockdividendspersharecashpaid"]

In [8]:
included_columns=[
    "year",
    "quarter",
    "week",
    "ticker",
    "adjclose"
]
included_columns.extend(financial_factors)
included_columns.extend(dividend_factors)
included_columns = list(set(included_columns))

In [9]:
training_sets = []
sec.connect()
market.connect()
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        filing = sec.retrieve_filing_data(cik)
        filing = p.column_date_processing(filing)
        ticker_data = financial.transform(prices,filing)
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
sec.disconnect()
market.disconnect()
data = pd.concat(training_sets)

  1%|█▉                                                                                                                                                        | 6/490 [00:01<01:53,  4.25it/s]

"['entitycommonstocksharesoutstanding'] not in index"


  2%|██▌                                                                                                                                                       | 8/490 [00:02<02:16,  3.53it/s]

"['stockholdersequity'] not in index"


  4%|█████▉                                                                                                                                                   | 19/490 [00:05<02:36,  3.01it/s]

...

 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████     | 474/490 [02:46<00:07,  2.09it/s]

"['propertyplantandequipmentnet'] not in index"


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 489/490 [02:52<00:00,  1.90it/s]

"['cashandcashequivalentsatcarryingvalue'] not in index"


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [02:52<00:00,  2.84it/s]


In [10]:
prediction_slice = data[data["year"]>=start_date.year-2].copy().reset_index(drop=True)
prediction_slice_quarterly = prediction_slice.copy().groupby(["year","quarter","ticker"]).mean().reset_index()

In [11]:
data.sort_values(["year","quarter","week"])[["year","quarter","week"]]

,year,quarter,week
0,2000,1,7.047619
0,2000,1,7.047619
0,2000,1,7.047619
0,2000,1,7.047619
0,2000,1,7.047619
...,...,...,...
46,2023,2,17.589744
93,2023,2,17.589744
93,2023,2,17.589744
87,2023,2,17.589744


In [12]:
fin_db.connect()
financial_models = fin_db.retrieve("models")
fin_db.disconnect()

In [13]:
earnings_db.connect()
earnings_models = earnings_db.retrieve("models")
earnings_db.disconnect()

In [14]:
dividends_db.connect()
dividends_models = dividends_db.retrieve("models")
dividends_db.disconnect()

In [15]:
financial_simulation = umod.recommend(financial_models,prediction_slice_quarterly.copy(),financial_factors)
financial_simulation["financial_prediction"] = (financial_simulation["cat_prediction"] + financial_simulation["xgb_prediction"]) / 2
fin_db.connect()
fin_db.drop("predictions")
fin_db.store("predictions",financial_simulation)
fin_db.disconnect()

In [18]:
earnings_simulation = umod.recommend(earnings_models,prediction_slice_quarterly.copy(),financial_factors)
earnings_simulation["earnings_prediction"] = (earnings_simulation["cat_prediction"] + earnings_simulation["xgb_prediction"]) / 2
earnings_db.connect()
earnings_db.drop("predictions")
earnings_db.store("predictions",earnings_simulation)
earnings_db.disconnect()

In [19]:
dividends_simulation = umod.recommend(dividends_models,prediction_slice_quarterly.copy(),dividend_factors)
dividends_simulation["dividends_prediction"] = (dividends_simulation["cat_prediction"] + dividends_simulation["xgb_prediction"]) / 2
dividends_db.connect()
dividends_db.drop("predictions")
dividends_db.store("predictions",dividends_simulation)
dividends_db.disconnect()